In [27]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [28]:
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Punta Arenas,CL,-53.15,-70.92,39.20,86,100,3.36,overcast clouds,0.0,0.0
1,1,Kapaa,US,22.08,-159.32,78.80,74,75,16.11,broken clouds,0.0,0.0
2,2,Sambava,MG,-14.27,50.17,73.13,77,1,14.18,clear sky,0.0,0.0
3,3,Casas Grandes,MX,30.37,-107.95,69.44,24,3,8.75,clear sky,0.0,0.0
4,4,Agdam,AZ,39.99,46.93,66.56,63,9,3.36,clear sky,0.0,0.0


In [29]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [30]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [31]:
pcities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [32]:
rain_p = str(input("Do you want it to be raining? (yes/no) "))
snow_p = str(input("Do you want it to be snowing? (yes/no) "))

Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [33]:
if (rain_p == 'yes'):
    pcities_rp_df = pcities_df.loc[(pcities_df["Rain (inches)"] > 0)]
    
elif (rain_p == 'no'):
    pcities_rp_df = pcities_df.loc[(pcities_df["Rain (inches)"] == 0)]
    
if (snow_p == 'yes'):
    pcities_rp_sp_df = pcities_rp_df.loc[(pcities_rp_df["Snow (inches)"] > 0)]
    
elif (snow_p == 'no'):
    pcities_rp_sp_df = pcities_rp_df.loc[(pcities_rp_df["Snow (inches)"] == 0)]


In [34]:
cities_w_rain_snow = len(pcities_df.loc[(pcities_df["Rain (inches)"] > 0)]) + len(pcities_rp_df.loc[(pcities_rp_df["Snow (inches)"] > 0)])
print("There are " + str(cities_w_rain_snow) + " cities with rain and snow")

There are 48 cities with rain and snow


In [35]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pcities_rp_sp_df[["City", "Country", "Max Temp","Current Description","Lat","Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,78.80,broken clouds,22.08,-159.32,
2,Sambava,MG,73.13,clear sky,-14.27,50.17,
3,Casas Grandes,MX,69.44,clear sky,30.37,-107.95,
4,Agdam,AZ,66.56,clear sky,39.99,46.93,
6,Mahebourg,MU,73.40,shower rain,-20.41,57.70,
8,Paita,PE,69.62,overcast clouds,-5.09,-81.11,
10,Puerto Ayora,EC,73.99,scattered clouds,-0.74,-90.35,
11,Rikitea,PF,76.05,scattered clouds,-23.12,-134.97,
13,Busselton,AU,66.00,overcast clouds,-33.65,115.33,
15,Geraldton,AU,69.80,broken clouds,-28.77,114.60,


In [36]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [44]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [58]:
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,78.80,broken clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
2,Sambava,MG,73.13,clear sky,-14.27,50.17,Melrose
3,Casas Grandes,MX,69.44,clear sky,30.37,-107.95,PUEBLO DEL SOuL
6,Mahebourg,MU,73.40,shower rain,-20.41,57.70,Shandrani Beachcomber Resort & Spa
8,Paita,PE,69.62,overcast clouds,-5.09,-81.11,Hotel Náutico de Paita


In [59]:
# Create the output file (CSV).
output_data_file2 = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file2, index_label="City_ID")

In [60]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [61]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [62]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
#max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, -80), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

248